In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '/home/jw3514/Work/CellType_Psy/src')
from CellType_PSY import *
#import scanpy as sc
HGNC, ENSID2Entrez, GeneSymbol2Entrez, Entrez2Symbol = LoadGeneINFO()

# Preprocessing

In [ ]:
def Z1Conversion(ExpMat, outname="test.z1.mat"):
    Z_mat = []
    for g, row in ExpMat.iterrows():
        tmp = ZscoreConverting(row.values)
        Z_mat.append(tmp)
    Z_mat = np.array(Z_mat)
    CT_Z1_DF = pd.DataFrame(data=Z_mat, index=ExpMat.index.values, 
                            columns=ExpMat.columns.values)
    CT_Z1_DF.to_csv(outname)
    return CT_Z1_DF

## Cell Type

In [ ]:
MedianDF = pd.read_csv("dat/SC_UMI_Mats/cluster_MeanLogUMI.csv", index_col=0)
MedianDF_Z1 = Z1Conversion(MedianDF, "dat/SC_UMI_Mats/cluster_V3_Z1Mat.csv")

In [ ]:
max_Z, min_Z = 5, -5
MedianDF_Z1_clipped = MedianDF_Z1.clip(upper=max_Z, lower=min_Z)
MedianDF_Z1_clipped.to_csv("dat/SC_UMI_Mats/cluster_V3_Z1Mat.clip.csv")

In [ ]:
max_Z, min_Z = 3, -3
MedianDF_Z1_clipped3 = MedianDF_Z1.clip(upper=max_Z, lower=min_Z)
MedianDF_Z1_clipped3.to_csv("dat/SC_UMI_Mats/cluster_V3_Z1Mat.clip3.csv")

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/ClusterV3_UMI_ISHMatch_Z2/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/SC_UMI_Mats//Cluster_Z2Mat_ISHMatch.csv")

In [ ]:
# Combine Zclip3
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/ClusterV3_UMI_ISHMatch_z1cplip3_Z2//"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/SC_UMI_Mats//Cluster_Z2Mat_ISHMatch.z1clip3.csv")

In [ ]:
Z2_Z1clip5 = pd.read_csv("dat/SC_UMI_Mats//Cluster_Z2Mat_ISHMatch.csv", index_col=0)
Z2_Z1clip3 = pd.read_csv("dat/SC_UMI_Mats//Cluster_Z2Mat_ISHMatch.z1clip3.csv", index_col=0)

In [ ]:
genes = Z2_Z1clip3.index.values

In [ ]:
Z2_Z1clip5.head(2)

In [ ]:
ct = "0001 CLA-EPd-CTX Car3 Glut_1"
X = Z2_Z1clip5.loc[genes, ct]
Y = Z2_Z1clip3.loc[genes, ct]

In [ ]:
plt.scatter(X,Y,s=0.5)
plt.plot([-5,8], [-5, 8], color="grey")
plt.grid(True)

In [ ]:
sns.kdeplot(X)
sns.kdeplot(Y)

##### Test Check Z1 cal

In [ ]:
sys.path.insert(1, '/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/scripts')
from script_Z2_calculation import *

In [ ]:
class fakeargs:
    def __init__(self, Z1mat, matchDIR):
        self.input = Z1mat
        self.match = matchDIR
        self.start = 1
        self.step = 1
        self.outdir = ""

In [ ]:
Z1mat = "dat/SC_UMI_Mats/cluster_V3_Z1Mat.clip.csv"
matchDIR = "/home/jw3514/Work/ASD_Circuits/dat/genes/ExpMatch_RootExp_uniform_kernal/"

In [ ]:
args = fakeargs(Z1mat, matchDIR)
ins = script_Z2_calculation(args)

In [ ]:
entrez=1
match_genes = loadgenelist(ins.matchDir+"/{}.csv".format(entrez), toint=True)
match_genes = [int(g) for g in match_genes if g in ins.z1_mat.index.values]
match_genes = match_genes[:1000]

In [ ]:
STR = "0494 OB Eomes Ms4a15 Glut_4"
gene = entrez
z_gene = ins.z1_mat.loc[gene, STR]
z_matches = [ins.z1_mat.loc[g, STR] for g in match_genes]
z_matches2 = [x for x in z_matches if x == x]
b = (z_gene - np.mean(z_matches2)) / np.std(z_matches2)

In [ ]:
print(len(match_genes), len(z_matches), len(z_matches2))

In [ ]:
z_matches22 = [(g,x) for (g,x) in zip(match_genes, z_matches) if x == x]

In [ ]:
with open("Entrez_1_CT_0494_MatchZ1.txt", "wt") as f:
    for g,z in z_matches22:
        f.write("{},{}\n".format(g,z))

In [ ]:
print(b)

In [ ]:
print(z_gene)
print(np.mean(z_matches), np.std(z_matches))

In [ ]:
print(len(z_matches))

In [ ]:
np.nan == np.nan

## MERFISH

In [ ]:
def quantile_normalize(df):
    ranks = df.rank(method='min', na_option='keep')
    sorted_df = np.sort(df.values, axis=0)
    nan_mask = np.isnan(df.values)
    sorted_df[nan_mask] = 0
    means = np.nanmean(np.where(nan_mask, np.nan, sorted_df), axis=1)
    means[np.isnan(sorted_df).all(axis=1)] = np.nan
    rank_mean_dict = {rank: mean for rank, mean in zip(np.arange(1, len(means)+1), means)}
    normalized_df = ranks.applymap(lambda x: rank_mean_dict.get(x, np.nan))

    return normalized_df

In [ ]:
MERFISH_STRAnn = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/MERFISH.ISH_Annot.csv", index_col=0)

In [ ]:
## Annotate EXP: script_STR_SC_ExpMat.py

In [ ]:
MERFISH_CellMeanExp = pd.read_csv("dat/MERFISH/STR_Cell_Mean_DF.UMI.csv", index_col=0)
MERFISH_VolMeanExp = pd.read_csv("dat/MERFISH/STR_Vol_Mean_DF.UMI.csv", index_col=0)

In [ ]:
MERFISH_CellMean_Z1 = Z1Conversion(MERFISH_CellMeanExp, "dat/MERFISH/STR_Cell_Mean_Z1Mat.csv")
MERFISH_VolMean_Z1 = Z1Conversion(MERFISH_VolMeanExp, "dat/MERFISH/STR_Vol_Mean_Z1Mat.csv")

In [ ]:
max_Z, min_Z = 5, -5
DF_Z1_clipped = MERFISH_CellMean_Z1.clip(upper=max_Z, lower=min_Z)
DF_Z1_clipped.to_csv("dat/MERFISH/STR_Cell_Mean_Z1Mat.clip.csv")

max_Z, min_Z = 5, -5
DF_Z1_clipped = MERFISH_VolMean_Z1.clip(upper=max_Z, lower=min_Z)
DF_Z1_clipped.to_csv( "dat/MERFISH/STR_Vol_Mean_Z1Mat.clip.csv")

In [ ]:
MERFISH_VolMeanExp_EXP_QN = quantile_normalize(MERFISH_VolMeanExp)
MERFISH_VolMeanExp_QN_Z1 = Z1Conversion(MERFISH_VolMeanExp_EXP_QN, "dat/MERFISH/STR_Vol_Mean_QNZ1Mat.csv")

In [ ]:
MERFISH_CellMeanExp = pd.read_csv("dat/MERFISH_Zhuang/STR_Cell_Mean_DF.UMI.csv", index_col=0)
MERFISH_VolMeanExp = pd.read_csv("dat/MERFISH_Zhuang/STR_Vol_Mean_DF.UMI.csv", index_col=0)

In [ ]:
MERFISH_CellMean_Z1 = Z1Conversion(MERFISH_CellMeanExp, "dat/MERFISH_Zhuang/STR_Cell_Mean_Z1Mat.csv")
MERFISH_VolMean_Z1 = Z1Conversion(MERFISH_VolMeanExp, "dat/MERFISH_Zhuang/STR_Vol_Mean_Z1Mat.csv")

In [ ]:
max_Z, min_Z = 5, -5
DF_Z1_clipped = MERFISH_CellMean_Z1.clip(upper=max_Z, lower=min_Z)
DF_Z1_clipped.to_csv("dat/MERFISH_Zhuang/STR_Cell_Mean_Z1Mat.clip.csv")

max_Z, min_Z = 5, -5
DF_Z1_clipped = MERFISH_VolMean_Z1.clip(upper=max_Z, lower=min_Z)
DF_Z1_clipped.to_csv( "dat/MERFISH_Zhuang/STR_Vol_Mean_Z1Mat.clip.csv")

In [ ]:
MERFISH_NEU_MeanExp = pd.read_csv("dat/MERFISH/STR_NEU_Mean_DF.UMI.csv", index_col=0)

In [ ]:
MERFISH_NEU_Z1 = Z1Conversion(MERFISH_NEU_MeanExp, "dat/MERFISH/STR_NEU_Mean_Z1Mat.csv")
max_Z, min_Z = 5, -5
DF_Z1_clipped = MERFISH_NEU_Z1.clip(upper=max_Z, lower=min_Z)
DF_Z1_clipped.to_csv("dat/MERFISH/STR_NEU_Mean_Z1Mat.clip.csv")

In [ ]:
MERFISH_NEU_Vol_MeanExp = pd.read_csv("dat/MERFISH/STR_NEU_Vol_Mean_DF.UMI.csv", index_col=0)
MERFISH_NEU_Vol_Z1 = Z1Conversion(MERFISH_NEU_Vol_MeanExp, "dat/MERFISH/STR_NEU_Vol_Mean_Z1Mat.csv")
max_Z, min_Z = 5, -5
DF_Z1_clipped = MERFISH_NEU_Vol_Z1.clip(upper=max_Z, lower=min_Z)
DF_Z1_clipped.to_csv("dat/MERFISH/STR_NEU_Vol_Mean_Z1Mat.clip.csv")

In [ ]:
MERFISH_NEU_Vol_MeanExp.head(2)

In [ ]:
MERFISH_CellMeanExp_EXP_QN = quantile_normalize(MERFISH_CellMeanExp)
MERFISH_CellMeanExp_QN_Z1 = Z1Conversion(MERFISH_CellMeanExp_EXP_QN, "dat/MERFISH/STR_Cell_Mean_QNZ1Mat.csv")

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/MERFISH_Allen_CellMean_UMI_ISHMatch_Z2/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/MERFISH/STR_Cell_Mean_Z2Mat_ISHMatch.csv")

Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/MERFISH_Allen_VolMean_UMI_ISHMatch_Z2/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/MERFISH/STR_Vol_Mean_Z2Mat_ISHMatch.csv")

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/MERFISH_Allen_NEU_Mean_UMI_ISHMatch_Z2/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/MERFISH/STR_NEUR_Mean_Z2Mat_ISHMatch.csv")

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/MERFISH_Allen_NEU_Vol_Mean_UMI_ISHMatch_Z2/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/MERFISH/STR_NEUR_Vol_Mean_Z2Mat_ISHMatch.csv")

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/MERFISH_MIT_CellMean_UMI_ISHMatch_Z2/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/MERFISH_Zhuang/STR_Cell_Mean_Z2Mat_ISHMatch.csv")

Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/MERFISH_MIT_VolMean_UMI_ISHMatch_Z2/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("dat/MERFISH_Zhuang/STR_Vol_Mean_Z2Mat_ISHMatch.csv")

#### Experssion Matching Based on MERFISH

In [ ]:
MERFISH_STRAnn = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/MERFISH.ISH_Annot.csv", index_col=0)

In [ ]:
ClusterExpDF = pd.read_csv("dat/SC_UMI_Mats/cluster_MeanLogUMI.csv", index_col=0)

In [ ]:
Total_Exp_Genes = np.zeros(ClusterExpDF.shape[0])
for c, row in MERFISH_STRAnn.iterrows():
    cluster = row["cluster"]
    if cluster in ClusterExpDF.columns.values:
        exp = ClusterExpDF[cluster]
        Total_Exp_Genes += exp

In [ ]:
Total_Exp_Genes.index

In [ ]:
WB_ExpDF = pd.DataFrame(Total_Exp_Genes)
WB_ExpDF.columns=["TotalExp"]

In [ ]:
WB_ExpDF = WB_ExpDF.sort_values("TotalExp")
WB_ExpDF["Rank"] = [1+x for x in range(WB_ExpDF.shape[0])] # compute Rank
WB_ExpDF["quantile"] = WB_ExpDF["Rank"]/WB_ExpDF.shape[0]
WB_ExpDF.to_csv("MouseMERFISHGeneMatchQuantile.csv")

In [ ]:
WB_ExpDF